In [8]:
import pandas as pd
import re
import matplotlib.pyplot as plt

from maple.types import Action, ActionType
from maple.classification import parse

import warnings
warnings.filterwarnings('ignore')

from dateutil.parser import parse
from datetime import datetime

pd.options.display.max_colwidth = -1


In [9]:
# committees

# TO DO check and complete these lists - update in january 
first_committees  = [
  "advanced information technology, the internet and cybersecurity",
  "bonding, capital expenditures and state assets", "cannabis policy" ,
  "children, families and persons with disabilities",
  "community development and small businesses",
  "consumer protection and professional licensure",
  "covid-19 and emergency preparedness and management",
  "economic development and emerging technologies",
  "higher education",
  "education",
  "elder affairs",
  "election laws",
  "environment, natural resources and agriculture",
  "export development",
  "financial services",
  "Housing and Food Security", 
  "Housing and Local Aid Social Services and Veteran Services and Soldiers",
  "Housing Energy and Environmental Affairs and com_name"
  "housing",
  "the judiciary",
  "labor and workforce development",
  "mental health substance use and recovery",
  "municipalities and regional government",
  "public health",
  "public safety and homeland security",
  "public service",
  "racial equity, civil rights and inclusion ",
  "revenue",
  "state administration and regulatory oversight",
  "telecommunications, utilities and energy",
  "tourism, arts and cultural development",
  "transportation",
  "Veterans and Federal Affairs",
  "House Bills in Third",
  "Election Laws",
  "Transportation",
  ]


chamber_committees = [
  "committee on rules",
  'committee on rules of the two branches, acting concurrently',
  "committee on \w* ways and means", 
  "committee on \w* steering, policy and scheduling",
  "committee on \w* steering and policy",
  "committee on \w* housing and finance",
  "committee on health care financing",
  "committee on bills in the third reading"
]


def has_first_committees(action):
  for f in first_committees:
    m = re.compile(f, re.I).search(action)
    if m != None: 
      return True
  return False

def has_chamber_committees(action):
  for c in chamber_committees:
    m = re.compile(c , re.I).search(action)
    if m != None: 
     return True
  return False


In [10]:
labels = [  
  ('orders of the day', 'first_chamber'),
  ('reported', 'first_chamber'),
  ('engrossed', 'second_chamber'),
  ('signed by the governor', 'signed by the governor'),
]

label_order = ('no status','first_committee', 'chamber_committee', 'first_chamber', 'second_chamber', 'signed by the governor')


def get_labels(action):
  acts = 0
  if has_first_committees(action): 
    acts = 1
  if has_chamber_committees(action):
    acts = 2
  for keyword, label in labels: 
    if re.compile(keyword, re.I).search(action) is not None:
      if label in label_order and label_order.index(label) > acts:
        acts = label_order.index(label)
  return acts
  
      


In [11]:
# input from csv and run label finder

df = pd.read_csv('./analysis/data/all-history-actions.csv')
df['action_id'] = df.index

df['date'] = df.date.apply(lambda x: parse(x))

df['stage'] = df['action'].apply(lambda s: label_order[get_labels(s)])



df


,id,action,branch,date,action_id,stage
0,H1,Referred to the committee on House Ways and Means,House,2021-02-25 16:46:19.933000+00:00,0,chamber_committee
1,H1,"Reported, in part, by H4000",House,2021-04-15 04:00:00+00:00,1,first_chamber
2,H10,Referred to the committee on Public Service,House,2021-04-13 15:59:37.223000+00:00,2,first_committee
3,H10,Senate concurred,Senate,2021-04-13 15:59:37.223000+00:00,3,no status
4,H10,Hearing scheduled for 07/28/2021 from 01:00 PM-04:00 PM in Virtual Hearing,Joint,2021-07-22 14:53:10.290000+00:00,4,no status
...,...,...,...,...,...,...
45467,s2729,Recommended new text for H4126,Senate,2022-02-28 05:00:00+00:00,45467,no status
45468,s2729,Rules suspended,Senate,2022-02-28 05:00:00+00:00,45468,no status
45469,s2729,Substituted as a new text for H4126,Senate,2022-02-28 05:00:00+00:00,45469,no status
45470,s2729,See H4126,Senate,2022-02-28 05:00:00+00:00,45470,no status
